In [ ]:
#@title Load data { form-width: "10%" }

!cp ./drive/MyDrive/tmp/scl-2021-ds.zip ./
!unzip ./scl-2021-ds.zip

Archive:  ./scl-2021-ds.zip
  inflating: sampleSubmission.csv    
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
#@title install packages { form-width: "10%" }
!pip install torch==1.8.0
!pip install transformers
!pip install simpletransformers

In [ ]:
#@title preprocess { form-width: "10%" }

import pandas as pd
train_df = pd.read_csv("./train.csv")

import random
labels = ['B-STR', 'I-STR', 'B-POI', 'I-POI', 'O']

def address2label(train, val, address, POI_and_street):
    # val_ratio = 0.2
    # if random.random() < val_ratio:
    #     f = val
    # else:
    #     f = train
    f = train

    # skip date that the answer can't be copy from address
    POI, street = POI_and_street.split('/')
    if POI not in address or street not in address:
        return
    
    address_first_word = ''.join([word[0] for word in address.split()])
    POI_first_word = ''.join([word[0] for word in POI.split()])
    street_first_word = ''.join([word[0] for word in street.split()])
    
    try:
        POI_start_i = address_first_word.index(POI_first_word)
        street_start_i = address_first_word.index(street_first_word)
    except:
        return
    
    POI_len = len(POI_first_word)
    street_len = len(street_first_word)
    
    address_list = address.split()
    for i in range(len(address_list)):
        try:
            if i == POI_start_i and len(POI):
                f.write(f'{address_list[i]}\tB-POI\n')
            elif i > POI_start_i and i < POI_start_i + POI_len:
                f.write(f'{address_list[i]}\tI-POI\n')
            elif i == street_start_i and len(street):
                f.write(f'{address_list[i]}\tB-STR\n')
            elif i > street_start_i and i < street_start_i + street_len:
                f.write(f'{address_list[i]}\tI-STR\n')
            else:
                f.write(f'{address_list[i]}\tO\n')
        except:
            print(address_list[i])
    f.write('\n')
    
with open('train_complete.txt', 'w', encoding='utf-8') as train, open('val_complete.txt', 'w', encoding='utf-8') as val:
    train_df.apply(lambda x: address2label(train, val, x['raw_address'], x['POI/street']), axis=1)


# Function to read ner file in connl format and return a DataFrame with columns: sentence_id, words, labels
def get_ner_data(filename):
    word_list = []
    sentence_counter = 0
    with open(filename) as fp:
        for cnt, line in enumerate(fp):
            try:
                texts = line.split()
                if len(texts) != 0:
                    word, label = ' '.join(texts[0:-1]), texts[-1]
                    word_list.append([sentence_counter, word, label])
                else:
                    sentence_counter += 1
            except:
                print("Unexpected error:", sys.exc_info()[0], cnt, line)
                word_list.append([sentence_counter, "", ""])              
                sentence_counter += 1
                pass
    print(f'read {sentence_counter} lines')
    ner_data = pd.DataFrame(word_list, columns=["sentence_id", "words", "labels"])
    return ner_data

In [ ]:
#@title training { form-width: "10%" }
from simpletransformers.ner import NERModel, NERArgs

model_args = NERArgs()
model_args.num_train_epochs = 2
model_args.train_batch_size = 64
model_args.evaluate_during_training = False
model_args.output_dir = f'output'
model_args.best_model_dir = f'best_model'
model_args.fp16 = False
model_args.labels_list = labels
model_args.do_lower_case = True
model_args.overwrite_output_dir = True

# model_save_steps = -1
# model_save_eval_checkpoints = False

# model = NERModel(
#     "xlmroberta", "cahya/xlm-roberta-large-indonesian-NER", labels=labels, args=model_args
# )

model = NERModel(
    "bert", "cahya/bert-base-indonesian-522M", labels=labels, args=model_args
)

train_data = get_ner_data("./train_complete.txt")
val_data = get_ner_data("./val_complete.txt")
# train_data = "./train_complete.txt"
# val_data = "./val_complete.txt"
model.train_model(train_data, eval_data=val_data)

Some weights of the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesia

read 239779 lines
read 0 lines


  0%|          | 0/3 [00:02<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/3747 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/3747 [00:00<?, ?it/s]

(7494, 0.25534093440620936)

In [ ]:
#@title output { form-width: "10%" }

test_df = pd.read_csv("test.csv")
test_data = list(test_df['raw_address'])

predictions, raw_outputs = model.predict(test_data)
# predictions, raw_outputs = model.preedict("val_complete.txt")

with open('output_no_comma_bert.csv', 'w', encoding='utf-8') as f:
  f.write('id,POI/street\n')
  for i, pred in enumerate(predictions):
    POI = []
    street = []
    for word_dict in pred:
      for word, label in word_dict.items():
        if label in ['B-POI', 'I-POI']:
          POI.append(word)
        elif label in ['B-STR', 'I-STR']:
          street.append(word)
    # out_POI = ' '.join(POI)
    # out_street = ' '.join(street)
    # f.write(f'{i},"{out_POI}/{out_street}"\n')
    out_POI = ' '.join(POI).replace(',', '')                  # 不顧一切去掉逗號，可刪
    out_street = ' '.join(street).replace(',', '')
    f.write(f'{i},{out_POI}/{out_street}\n')

  0%|          | 0/2 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/6250 [00:00<?, ?it/s]

In [ ]:
from google.colab import files

files.download('output_no_comma_bert.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cp -r best_model/ '/content/drive/MyDrive/tmp/'